In [1]:
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('./data/merged.csv')
df = df[['Singer', 'Name', 'Lyric', 'Category']].astype(str)
df.drop(df[df.Singer == 'nan'].index, inplace=True)
df.drop(df[df.Name == 'Name'].index, inplace=True)
print("\033[32mTotal number of data: %d\033[0m" %len(df))

Total number of data: 2206


In [3]:
d = {'Category': df['Category'].value_counts().index, 'count': df['Category'].value_counts()}
df_cat = pd.DataFrame(data = d).reset_index(drop = True)
df_cat

,Category,count
0,愛情,874
1,想法表達,405
2,悲傷,216
3,反抗反駁反諷,206
4,勵志,167
5,嗨歌,125
6,離別,76
7,友情,42
8,特殊主題,39
9,親情,33


In [4]:
df.loc[df.Category == '勵志','Category'] = '勵志友情'
df.loc[df.Category == '友情','Category'] = '勵志友情'
df.loc[df.Category == '悲傷','Category'] = '悲傷離別'
df.loc[df.Category == '離別','Category'] = '悲傷離別'
df.loc[df.Category == '特殊主題','Category'] = '其他'
df.loc[df.Category == '嗨歌','Category'] = '其他'
df.loc[df.Category == '親情','Category'] = '其他'
df.loc[df.Category == '感謝','Category'] = '想法表達'

In [5]:
d = {'Category': df['Category'].value_counts().index, 'count': df['Category'].value_counts()}
df_cat = pd.DataFrame(data = d).reset_index(drop = True)
df_cat

,Category,count
0,愛情,874
1,想法表達,428
2,悲傷離別,292
3,勵志友情,209
4,反抗反駁反諷,206
5,其他,197


In [6]:
condition = df['Category'] == '愛情'
df_select = df[condition].sample(200)
condition = df['Category'] == '想法表達'
df_tmp = df[condition].sample(0)
df_select = pd.concat([df_select, df_tmp])
condition = df['Category'] == '勵志友情'
df_tmp = df[condition].sample(200)
df_select = pd.concat([df_select, df_tmp])
condition = df['Category'] == '悲傷離別'
df_tmp = df[condition].sample(200)
df_select = pd.concat([df_select, df_tmp])
condition = df['Category'] == '反抗反駁反諷'
df_tmp = df[condition].sample(200)
df_select = pd.concat([df_select, df_tmp])

df_select = df_select.reset_index(drop=True)

In [7]:
d = {'Category': df_select['Category'].value_counts().index, 'count': df_select['Category'].value_counts()}
df_cat = pd.DataFrame(data = d).reset_index(drop = True)
df_cat

,Category,count
0,愛情,200
1,勵志友情,200
2,悲傷離別,200
3,反抗反駁反諷,200


In [8]:
categories = list(df_select['Category'].value_counts().index)
cat_to_id = dict(zip(categories, range(len(categories))))
def label_to_id(line):
    line = str(line)
    return cat_to_id[line]

In [9]:
def remove_punctuation(line):
    stopwords = [line.strip() for line in open('./data/stopwords.txt', 'r', encoding='utf-8').readlines()]
    line = str(line)
    if line.strip() == '':
        return ''
    re_han = re.compile(u"[^a-zA-Z0-9\u4E00-\u9FA5]")
    line = re_han.sub('', line)
    cut = [w for w in list(jieba.cut(line)) if w not in stopwords]
    res = " ".join(cut)
    return res

In [10]:
import re
import jieba
df_select['y'] = df_select['Category'].apply(label_to_id)
df_select['cut_text_string'] = df_select['Lyric'].apply(remove_punctuation)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.889 seconds.
Prefix dict has been built successfully.


In [11]:
df_select

,Singer,Name,Lyric,Category,y,cut_text_string
0,張惠妹,讓你飛,等你一整夜 雨也飄一整夜我濕透的愛 凌亂不堪今夜你多情的胸膛 在哪裡遊蕩越想抓緊你 你越逃越...,愛情,0,一整夜 雨 飄 一整夜 濕透 愛凌亂 不堪 今夜 多情 胸膛 哪裡遊 蕩 越 想 抓緊 越 ...
1,楊丞琳,觀眾,對於愛情 一直都關乎我的姓名愛你不愛你 越來越不要緊確實戴了墨鏡 卻想讓你靠近我想我還欠你一...,愛情,0,愛情 一直 關乎 姓名 愛你不愛你越 越 緊確 實戴 墨鏡 卻 想 靠近 想 還欠 一個 說...
2,梁靜茹,LOVE,LOVE 是他隨手扔的玩具散了一地從不整理 你丟我撿的遊戲LOVE 是他隨口聊的話題說了就忘...,愛情,0,LOVE 隨手 扔 玩具 散 一地 整理 丟我 撿 遊戲 LOVE 隨口 聊 話題 說 忘 ...
3,梁靜茹,愛久見人心,我冷漠 是不想被看出 太容易 被感動觸及我比較喜歡 現在的自己 不太想 回到過去我常常 為我...,愛情,0,冷漠 不想 看出 太 容易 感動觸 喜 歡現 不太想 回到 過去 之間 忽遠 忽近 的關 係...
4,林俊傑,只對你有感覺(Feel For You),無解的眼神 心像海底針光是猜測 我食慾不振有點煩人 又有點迷人浪漫沒天份 反應夠遲鈍不夠謹慎...,愛情,0,無解 眼神 心像 海底 針 猜測 食 慾 不振 有點 煩人 有點 迷人 浪漫 天份 反應夠 ...
...,...,...,...,...,...,...
795,田馥甄,人什麼的最麻煩了,你走下樓梯 在轉角最近的便利商店只是坐著 像不再期待地坐著手中那杯失溫的咖啡 比心還暖一點只...,反抗反駁反諷,3,走 下 樓梯 轉角 最近 便利商店 坐著 期待 坐 手中 那杯 失溫 咖啡 比心 還暖 一點...
796,陳奕迅,孤獨探戈,你可知道石頭 要幾多眼淚才被沖走你早知探戈 一下推一下卸便會失手你可知我背後 有幾多冷汗朝著...,反抗反駁反諷,3,知道 石頭 多眼 淚才 沖 走 早知 探戈 一下 推 一下 卸 失手 可知 背後有 幾多 冷...
797,張惠妹,母系社會,我不會 耕田吃草 讓人下注什麼理由 發明 什麼叫馬子難道是 想讓匹馬 為你生個兒子我沒有 兄...,反抗反駁反諷,3,耕田 吃 草 人 下注 理由 發明 叫馬子 想 匹馬為 生個 兒子 兄弟 聚會 幌子 煮 好...
798,陳奕迅,萬佛朝宗,香港的一粒星 初次出鏡想要紅北京的一粒沙 吹進一個溫暖胡同東京一位OL 想晉升也許成功伊拉克...,反抗反駁反諷,3,香港 一粒 星 初次 出鏡 想要 紅 北京 一粒 沙吹進 一個 溫暖 胡同 東京 一位 OL...


In [12]:
df_select = df_select.sample(frac = 1).reset_index(drop=True)
df_select

,Singer,Name,Lyric,Category,y,cut_text_string
0,田馥甄,諷刺的情書,愛過幾個人 篤定的眼神差點以為世界多天真糾纏幾回合 問自己要不要這麼笨屢次揮霍信任當夜讓房間...,反抗反駁反諷,3,愛過 幾個 人篤定 眼神 差點 世界 天真 糾纏 回合 問 笨 揮 霍 信任 當夜 房間 失...
1,A-Lin,笑笑的,想睡鬧鐘先按掉 心情不好要吃飽先讓脂肪飆一飆 才有機會變苗條煙味太重先落跑 車廂太擠就...,反抗反駁反諷,3,想 睡 鬧 鐘先 掉 心情 不好 吃 先 脂肪 飆一飆 機會 變苗條 煙味 太重 先落 跑 ...
2,五月天,錯錯錯,為什麼你能 同時那麼殘酷又溫柔 當你微笑告訴我 這是好結果一時之間我 不知道應該說些什麼 笑...,悲傷離別,2,殘酷 溫柔當 微笑 告訴 這是 好 一時 之間 知道 應該 說些 笑過 哭過 沒想過 最後要...
3,林俊傑,只對你有感覺(Feel For You),無解的眼神 心像海底針光是猜測 我食慾不振有點煩人 又有點迷人浪漫沒天份 反應夠遲鈍不夠謹慎...,愛情,0,無解 眼神 心像 海底 針 猜測 食 慾 不振 有點 煩人 有點 迷人 浪漫 天份 反應夠 ...
4,楊丞琳,小聰明,我不完美 偶而會皺眉難免小疲憊 迷上了咖啡有時傷悲 卻不再流淚依然愛玫瑰 卻不再曖昧Mayb...,愛情,0,完美 皺眉 難免 小 疲憊 迷上 咖啡 有時 傷悲 卻 流淚 依然 愛 玫瑰 卻 曖昧 Ma...
...,...,...,...,...,...,...
795,五月天,步步,空無一人的大街 闖入無人婚紗店 為你披上雪白誓言世界已灰飛煙滅 而愛矗立高樓間 你是真的或是...,愛情,0,空無一人 大街 闖入 無人 婚紗店 披上 雪白 誓言 世界 已灰飛 煙滅 愛 矗立 高樓 間...
796,陳奕迅,美麗謊言,只因不應該聽而聽 此刻不知怎樣去反應你問這事情 有沒有內情 這刻可否只當夢醒 只因不應該聽而...,悲傷離別,2,只 因不應 聽 聽 此刻 不知 反應 你問 事情 內情 這刻 可否 只 當夢醒 只 因不應 ...
797,陳奕迅,反高潮,這秒鐘 十分感動接下來 只可以更加激進如若要平庸 十一分的平庸微笑大笑後必須發瘋動不動 就很...,悲傷離別,2,這秒鐘 十分 感動 接下 只 激進 平庸 十一分 平庸 微笑 大笑 後 發瘋動 不動 感動 ...
798,林俊傑,丹寧執著,你的溫柔埋藏在我的脈搏連呼吸都能感動每一秒鐘 有你就有我的笑容汗水蒸發在這忙碌十字街頭藍色天...,勵志友情,1,溫柔 埋藏 脈 搏連 呼吸 感動 每一秒 鐘有 笑容 汗水 蒸發 忙碌 十字街 頭藍色 天 ...


# Build Model

In [13]:
'''# 計算詞頻
count_vect = CountVectorizer(stop_words='english', decode_error='ignore')
X_train_counts = count_vect.fit_transform(df_select.cut_text_string)
# 計算TF-IDF
tf_transformer = TfidfVectorizer(stop_words='english', decode_error='ignore')
X_train_counts_tf = tf_transformer.fit_transform(df_select.cut_text_string)
labels = df_select.Category'''

In [28]:
vectorizer = TfidfVectorizer()
X_train_counts_tf = vectorizer.fit_transform(df_select.cut_text_string)
labels = df_select.Category

In [47]:
import pickle
pickle.dump(vectorizer, open("./model/vectorizer.pickle", "wb"))

In [23]:
X_train_counts_tf

<800x18190 sparse matrix of type '<class 'numpy.float64'>'
	with 44925 stored elements in Compressed Sparse Row format>

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_train_counts_tf, labels, test_size = 0.2, random_state = 0, stratify = labels)

In [17]:
condition = df['Category'] == '想法表達'
df_think = df[condition].sample(200)
df_think = df_think.reset_index(drop=True)
df_think['cut_text_string'] = df_think['Lyric'].apply(remove_punctuation)

In [18]:
df_think

,Singer,Name,Lyric,Category,cut_text_string
0,田馥甄,獨善其身,眼睛 在忙著看風景嘴巴 想念著甜品這雙耳朵 只想保持安靜頭髮 在纏著它自己呼吸 在挑剔空氣這...,想法表達,眼睛 忙 風景 嘴巴 想念 甜品 這雙 耳朵 只想 保持 安靜頭 髮 纏 呼吸 挑剔 空氣 ...
1,蔡依林,開場白,再見面 已經是朋友了我們就這麼單純 瞎聊著彼此說以後時間過了 學會珍惜了原來最初的快樂 並不...,想法表達,再見面 已經 朋友 瞎聊 說 以後時 間過 學會 珍惜 原來 最初 快樂並 的終 能夠 坦白...
2,A-Lin,懂得,最近會有點忙 親愛的我想多虧你貼心的體諒自由自在飛翔 並非奢望心決定風向的晴朗有時候我在...,想法表達,最近 有點 忙 親愛的 想 虧 貼心 體諒 自由自在 飛翔 奢望 心決定 風向 晴朗 想 力...
3,SHE,不想長大,為什麼就是找不到 不謝的玫瑰花為什麼遇見的王子 都不夠王子啊我並不期盼他會有 玻璃鞋和白馬我...,想法表達,找 不到 不謝 玫瑰花 遇見 王子 不夠 王子 我並 期盼 他會 玻璃 鞋 白馬 驚訝 情話...
4,楊丞琳,青春鬥,時間喜歡在臉上胡亂塗鴉皺紋黑斑是它的左右護法想想辦法對付它 不管犧牲要多大女生一輩子都被時間...,想法表達,時間 喜歡 臉 胡亂 塗鴉 皺紋 黑斑 左右 護法 想想 辦法 對付 犧牲 女生 一輩子 時...
...,...,...,...,...,...
195,SHE,愛來過,我看不開 也放不開 因為我曾見過愛情真的盛開我要等待 一直等待 等那~一個夜晚從回憶回來當你...,想法表達,不開 放不開 曾 過愛情 真的 盛開 等待 一直 等待 一個 夜晚 回憶 回來當 擁抱 一瞬...
196,蔡依林,彩色相片(Color Photo),晴天 肆無忌憚的光線差些要模糊的臉 笑得很清淺轉眼 說不出聲的語言由生到死的纏綿 藏進心裡面...,想法表達,晴天 肆無忌憚 光線 差些 模糊 臉 笑 清 淺 轉眼 說 出聲 語言由生 死 纏 綿藏 進...
197,張惠妹,鳳凰于飛,柳媚花妍 鶯聲兒嬌 春色又向人間報到山眉水眼 盈盈的笑 我又投入了愛的懷抱像鳳凰于飛在雲霄 ...,想法表達,柳媚 花妍 鶯 聲兒 嬌 春色 人間 報到 山眉水眼 盈盈 笑 投入 愛的 懷抱 鳳凰于 飛...
198,林宥嘉,請說,坐著說 站著說 妳想要 說什麼音量怎麼這樣微弱 空氣都不振動 ha妳看我 看什麼 我不是 常...,想法表達,坐著 說 站 說 妳 想要 說 音量 微弱 空氣 不振 動 ha 妳 說 運動 減重 妳 再...


In [29]:
X_think = vectorizer.transform(df_think.cut_text_string)

In [30]:
X_think

<200x18190 sparse matrix of type '<class 'numpy.float64'>'
	with 7775 stored elements in Compressed Sparse Row format>

## Discriminative model

In [31]:
num_folds = 10
scoring = 'accuracy'

In [32]:
models = {}
models['LR'] = LogisticRegression() #逻辑回归
models['SVM'] = SVC() #支持向量机
models['CART'] = DecisionTreeClassifier() #分类与回归树
models['MNB'] = MultinomialNB() #朴素贝叶斯分类器
models['KNN'] = KNeighborsClassifier() #K近邻算法
results = []

for key in models:
    kfold = KFold(n_splits = num_folds)
    cv_results = cross_val_score(models[key], x_train, y_train, cv = kfold, scoring = scoring)
    results.append(cv_results)
    print('%s:%f(%f)' %(key,cv_results.mean(),cv_results.std()))

LR:0.496875(0.050775)
SVM:0.390625(0.062108)
CART:0.345313(0.041014)
MNB:0.420312(0.048639)
KNN:0.350000(0.050967)


In [33]:
# Logistic Regression
param_grid = {}
param_grid['C'] = list(range(0, 10, 2))
model = LogisticRegression()
kfold = KFold(n_splits = num_folds)
grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = scoring, cv = kfold)
grid_result = grid.fit(X = x_train, y = y_train)
print('best acc: %s with %s' % (grid_result.best_score_, grid_result.best_params_))

/home/vivian/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vivian/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vivian/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py", line 1614, in fit
    for class_, warm_start_coef_ in zip(classes_, warm_start_coef)
  File "/home/vivian/miniconda3/envs/tf2/lib/python3.7/site-packages/joblib/parallel

best acc: 0.490625 with {'C': 2}


In [19]:
# SVM
param_grid = {}
param_grid['C'] = list(range(0, 10, 2))
model = SVC()
kfold = KFold(n_splits = num_folds)
grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = scoring, cv = kfold)
grid_result = grid.fit(X = x_train, y = y_train)
print('best acc: %s with %s' % (grid_result.best_score_, grid_result.best_params_))

/home/vivian/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "/home/vivian/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/model_selection/_validation.py", line 681, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/vivian/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/svm/_base.py", line 255, in fit
    fit(X, y, sample_weight, solver_type, kernel, random_seed=seed)
  File "/home/vivian/miniconda3/envs/tf2/lib/python3.7/site-packages/sklearn/svm/_base.py", line 

best acc: 0.5265625 with {'C': 2}


In [20]:
# Naive Bayes
param_grid = {}
param_grid['alpha'] = [0.001, 0.01, 0.1, 1.0, 1.5, 2.0]
model = MultinomialNB()
kfold = KFold(n_splits = num_folds)
grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = scoring, cv = kfold)
grid_result = grid.fit(X = x_train, y = y_train)
print('best acc : %s with %s' % (grid_result.best_score_, grid_result.best_params_))

best acc : 0.4859375 with {'alpha': 0.1}


## Ensemble Learning

In [21]:
ensembles = {}
ensembles['RF'] = RandomForestClassifier()   # random forest
ensembles['AB'] = AdaBoostClassifier()       # Adaboost
results = []
for key in ensembles:
    kfold = KFold(n_splits = num_folds)
    cv_results = cross_val_score(ensembles[key], x_train, y_train, cv = kfold, scoring = scoring)
    results.append(cv_results)
    print('%s : %f (%f)' % (key, cv_results.mean(), cv_results.std()))

RF : 0.489063 (0.068839)
AB : 0.396875 (0.074346)


In [22]:
# Random Forest
param_grid = {}
param_grid['n_estimators'] = list(range(100, 200, 20))
param_grid['max_depth'] = list(range(10, 30, 5))
model = RandomForestClassifier()
kfold = KFold(n_splits = num_folds)
grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = scoring, cv = kfold)
grid_result = grid.fit(X = x_train, y = y_train)
print('best acc : %s with %s' % (grid_result.best_score_, grid_result.best_params_))

best acc : 0.5140625 with {'max_depth': 20, 'n_estimators': 180}


In [23]:
# AdaBoost
param_grid = {}
param_grid['n_estimators'] = list(range(100, 150, 20))
param_grid['learning_rate'] = [0.001, 0.01, 0.1, 0.5, 1]
model = AdaBoostClassifier()
kfold = KFold(n_splits = num_folds)
grid = GridSearchCV(estimator = model, param_grid = param_grid, scoring = scoring, cv = kfold)
grid_result = grid.fit(X = x_train, y = y_train)
print('best acc : %s with %s' % (grid_result.best_score_, grid_result.best_params_))

best acc : 0.415625 with {'learning_rate': 0.01, 'n_estimators': 140}


# Test the Final Model

In [24]:
'''model = LogisticRegression(C = 2)
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print(accuracy_score(list(y_test), list(predictions)))
print(classification_report(list(y_test), list(predictions)))'''

'model = LogisticRegression(C = 2)\nmodel.fit(x_train, y_train)\npredictions = model.predict(x_test)\nprint(accuracy_score(list(y_test), list(predictions)))\nprint(classification_report(list(y_test), list(predictions)))'

In [44]:
import joblib

In [45]:
model = LogisticRegression(C = 2)
model.fit(x_train, y_train)
joblib.dump(model, './model/LR_model')
# predictions = model.predict(X_think)

['./model/LR_model']

In [43]:
'''for lyric, label in zip(list(df_think['cut_text_string']), predictions):
    print(lyric)
    print(label)
    print("------------------------------------")'''

眼睛 忙 風景 嘴巴 想念 甜品 這雙 耳朵 只想 保持 安靜頭 髮 纏 呼吸 挑剔 空氣 腦袋 知道 微醺 道理 身體 髮 膚 天生 愛 有什麼 邏輯 想 愛別 人 運氣 先 善待 這身 體靈魂 再 相遇 先 善待 這顆 心 再 懂別 心 眼睛 風景 嘴巴 誰當 甜品 這雙 耳朵 安靜頭 髮 放開 呼吸 呼吸 腦袋 知道 微醺 道理 身體 髮 膚 自會 有靈 犀有 邏輯 想 愛的 人 愛 先 善待 這身 體靈魂 相遇 先 善待 這顆 心 懂 心 懂 心 懂 心 懂 心
愛情
------------------------------------
再見面 已經 朋友 瞎聊 說 以後時 間過 學會 珍惜 原來 最初 快樂並 的終 能夠 坦白 說 當時 脆弱 唯一 一次 不留 遺憾 簡單 結束 陌生 微笑 躲 這單 坦誠就 推翻 著不擾 笑 時間 停 珍惜 一刻 放下 累積 負荷 卸下 沉默 學會 保留 再見 後 真的 朋友 笑 以後時 間過 珍惜 原來 當時 快樂 記憶 中終 能夠 坦白 說 我現 寂寞 全新 一次 再也 遺憾 簡單 回應 傷痛 微笑 躲 這單 坦 誠 你好 多麼 默契 開場 笑 走 尷尬 回頭 湊 理由 希望 真的 快樂 說 好 說 走
勵志友情
------------------------------------
最近 有點 忙 親愛的 想 虧 貼心 體諒 自由自在 飛翔 奢望 心決定 風向 晴朗 想 力量 愛 放心 流浪 天上 風箏 自由 卻 不慌張 能夠 返航 夢 還握 手上 大雨 後 萌芽 土壤 再 疲憊 花盛開 後 芬芳 風 穿過 黑夜 星光 相逢 久 違 曙光 翅膀 陪 飛翔 天空 迷航 愛 最溫 暖 家 懂得 幸福 重量 珍貴 行囊 陪 達 那夢想 天堂 懂得 倔強 成全 天空 寬廣溫 柔 放 翱翔 勇敢 闖 接近 夢 遠方
勵志友情
------------------------------------
找 不到 不謝 玫瑰花 遇見 王子 不夠 王子 我並 期盼 他會 玻璃 鞋 白馬 驚訝 情話 會變 成謊 話 幸福 青鳥 飛得 高為 蘋果 擁抱 毒藥 沒想過 還孤單 可怕 突然 想起 從前 陪 洋娃娃 不想 不想 不想 長 大長 後 世界 沒童 話 不想 不想 不想 長 我寧願 永遠 笨 傻 不想 不想 不想 長 大長 後 

反抗反駁反諷
------------------------------------
看著 眼光 心腸 敢講 眼眶 翅膀 汪洋 衡量 悲傷 惆悵 奔忙 解放 冰霜 武裝 說法 體諒 太慌張 新 方向 新 地方 欲望 反光 新 天堂 莫名 緊張 圓些 謊 眼光 心腸 敢講 眼眶 翅膀 汪洋 衡量 悲傷 惆悵 奔忙 解放 冰霜 武裝 說法 體諒 太慌張 新 方向 新 地方 欲望 反光 新 天堂 莫名 緊張 圓些 謊 眼光 心腸 敢講 冰霜 武裝 說法 體諒
反抗反駁反諷
------------------------------------
時間 已經 忘記 有人 想著 回去 現在 是否 心情 喔 太美麗 玻璃瓶 裝著 人們 個性 一杯 一杯 所有 人 鼓起 勇氣 YA 沒忘記 喔 不敢 忘記 說 一字一句 喔 習慣 走 小路 不應 該還 遠現 覺得 一點 神秘 喔 好 美麗 風 景天 還有點 曬 阿試 圖 尋找 Green 色 門門 把手 喔 沒來 晚 不敢 來晚 要來 抓住 心 沒忘記 喔 不敢 忘記 說 一字一句 喔
悲傷離別
------------------------------------
知道 難過 愛一個 人 需要 緣份 越陷越深 傻 天真 碰觸 不安 靈魂 一天 只能 痴痴 愛一個 人別 太 認真 受傷 眼神 令人 心疼 一個 人 非要 一個 人才 能過 一生 逼 面對 傷痕 知道 難過 感情 付出 真心 別問 做 愛 長 久 道理 一天 懂 知道 難過 昨天 戀人 今天 說 分手 分手 別問 痛要 解脫 多情 人 註 傷 久 愛 人 太 認 真 受 傷 眼神 令人 心疼 一個 人 非要 一個 人才 能過 一生 逼 面對 傷痕 知道 難過 感情 付出 真心 別問 做 愛 長 久 道理 一天 懂 知道 難過 昨天 戀人 今天 說 分手 分手 別問 痛要 解脫 多情 人 註 傷 久 愛 變 成 刺 思念 成 癡 也許 心碎 愛情 最美 樣子 知道 難過 感情 付出 真心 別問 做 愛 長 久 道理 一天 懂 知道 難過 昨天 戀人 今天 說 分手 分手 別問 痛要 解脫 多情 人 註 傷 久
反抗反駁反諷
------------------------------------
有時 懂 黑 直到 水 被淹 沒到 眼前 有時 懂夜 直到 天 灑下 

In [25]:
'''# 設定字體
from matplotlib.font_manager import FontProperties
fonts = FontProperties(fname = "msjh.ttc", size = 14)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
from sklearn import metrics
import seaborn as sns
import numpy as np
confm = metrics.confusion_matrix(y_test, predictions)'''

'# 設定字體\nfrom matplotlib.font_manager import FontProperties\nfonts = FontProperties(fname = "msjh.ttc", size = 14)\n%config InlineBackend.figure_format = \'retina\'\n%matplotlib inline\nfrom sklearn import metrics\nimport seaborn as sns\nimport numpy as np\nconfm = metrics.confusion_matrix(y_test, predictions)'

In [26]:
'''# plot confusion matrix
plt.figure(figsize = (8, 8))
sns.heatmap(confm.T, square = True, annot = True,
            fmt = 'd', cbar = False,linewidths = .8,
            cmap = "YlGnBu")
plt.xlabel('True label', size = 14)
plt.ylabel('Predicted label', size = 14)
plt.xticks(np.arange(len(categories)) + 0.5, categories, fontproperties = fonts)
plt.yticks(np.arange(len(categories)) + 0.5, categories, fontproperties = fonts)
plt.show()'''

'# plot confusion matrix\nplt.figure(figsize = (8, 8))\nsns.heatmap(confm.T, square = True, annot = True,\n            fmt = \'d\', cbar = False,linewidths = .8,\n            cmap = "YlGnBu")\nplt.xlabel(\'True label\', size = 14)\nplt.ylabel(\'Predicted label\', size = 14)\nplt.xticks(np.arange(len(categories)) + 0.5, categories, fontproperties = fonts)\nplt.yticks(np.arange(len(categories)) + 0.5, categories, fontproperties = fonts)\nplt.show()'

In [27]:
'''model = SVC(C = 4)
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print(accuracy_score(list(y_test), list(predictions)))
print(classification_report(list(y_test), list(predictions)))'''

'model = SVC(C = 4)\nmodel.fit(x_train, y_train)\npredictions = model.predict(x_test)\nprint(accuracy_score(list(y_test), list(predictions)))\nprint(classification_report(list(y_test), list(predictions)))'

In [28]:
'''model = MultinomialNB(alpha = 1.5)
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print(accuracy_score(list(y_test), list(predictions)))
print(classification_report(list(y_test), list(predictions)))'''

'model = MultinomialNB(alpha = 1.5)\nmodel.fit(x_train, y_train)\npredictions = model.predict(x_test)\nprint(accuracy_score(list(y_test), list(predictions)))\nprint(classification_report(list(y_test), list(predictions)))'

In [29]:
'''model = RandomForestClassifier(max_depth = 25, n_estimators = 80)
# model = RandomForestClassifier()
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print(accuracy_score(list(y_test), list(predictions)))
print(classification_report(list(y_test), list(predictions)))'''

'model = RandomForestClassifier(max_depth = 25, n_estimators = 80)\n# model = RandomForestClassifier()\nmodel.fit(x_train, y_train)\npredictions = model.predict(x_test)\nprint(accuracy_score(list(y_test), list(predictions)))\nprint(classification_report(list(y_test), list(predictions)))'

In [30]:
'''model = AdaBoostClassifier(learning_rate = 0.5, n_estimators = 90)
model.fit(x_train, y_train)
predictions = model.predict(x_test)
print(accuracy_score(list(y_test), list(predictions)))
print(classification_report(list(y_test), list(predictions)))'''

'model = AdaBoostClassifier(learning_rate = 0.5, n_estimators = 90)\nmodel.fit(x_train, y_train)\npredictions = model.predict(x_test)\nprint(accuracy_score(list(y_test), list(predictions)))\nprint(classification_report(list(y_test), list(predictions)))'

In [31]:
'''y_pred = list(predictions)
y_test_t = list(y_test)'''

'y_pred = list(predictions)\ny_test_t = list(y_test)'

In [32]:
'''sent, label, predict = [], [], []
for i in range(len(y_pred)):
    if y_pred[i] != y_test_t[i]:
        sent.append(x_test[i])
        label.append(y_test_t[i])
        predict.append(y_pred[i])'''

'sent, label, predict = [], [], []\nfor i in range(len(y_pred)):\n    if y_pred[i] != y_test_t[i]:\n        sent.append(x_test[i])\n        label.append(y_test_t[i])\n        predict.append(y_pred[i])'